# Train/dev split

In [39]:
from sklearn.model_selection import train_test_split

In [79]:
texts, ids = [], []
with open('train_reviews.txt') as f:
    for line in f:
        text_id, text = line.rstrip('\r\n').split('\t')
        texts.append(text)
        ids.append(text_id)

In [80]:
train_texts, dev_texts, train_ids, dev_ids = train_test_split(texts, ids)

In [81]:
train_aspects, dev_aspects = [], []
with open('train_aspects.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_aspects.append(line)
        if text_id in dev_ids:
            dev_aspects.append(line)

In [82]:
train_sentiment, dev_sentiment = [], []
with open('train_cats.txt') as f:
    for line in f:
        line = line.rstrip('\r\n')
        text_id = line.split('\t')[0]
        if text_id in train_ids:
            train_sentiment.append(line)
        if text_id in dev_ids:
            dev_sentiment.append(line)

In [173]:
with open('train_split_aspects.txt', 'w') as f:
    for l in train_aspects:
        print(l, file=f)
with open('dev_aspects.txt', 'w') as f:
    for l in dev_aspects:
        print(l, file=f)
with open('train_split_reviews.txt', 'w') as f:
    for i, l in zip(train_ids, train_texts):
        print(i, l, sep="\t", file=f)
with open('dev_reviews.txt', 'w') as f:
    for i, l in zip(dev_ids, dev_texts):
        print(i, l, sep="\t", file=f)
with open('train_split_cats.txt', 'w') as f:
    for l in train_sentiment:
        print(l, file=f)
with open('dev_cats.txt', 'w') as f:
    for l in dev_sentiment:
        print(l, file=f)

# Baseline 1,2: категория и тональность упоминаний

Выделяем только аспекты, встретившиеся в train'е, приписываем самую частотную категорию.

In [85]:
import pandas as pd

In [86]:
train_asp = pd.read_csv(
    'train_split_aspects.txt', 
    delimiter='\t', 
    names=['text_id', 'category', 'mention', 'start', 'end', 'sentiment']
)
train_texts = pd.read_csv('train_split_reviews.txt', delimiter='\t', names=['text_id','text'])

In [105]:
import stanza
stanza.download('ru')

2021-12-15 10:57:15 INFO: Downloading default packages for language: ru (Russian)...
2021-12-15 10:57:17 INFO: File exists: /Users/rhubarb/stanza_resources/ru/default.zip.
2021-12-15 10:57:22 INFO: Finished downloading models and saved to /Users/rhubarb/stanza_resources.


In [107]:
nlp = stanza.Pipeline('ru', processors='tokenize,lemma')

2021-12-15 10:59:15 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| lemma     | syntagrus |

2021-12-15 10:59:15 INFO: Use device: cpu
2021-12-15 10:59:15 INFO: Loading: tokenize
2021-12-15 10:59:15 INFO: Loading: lemma
2021-12-15 10:59:15 INFO: Done loading processors!


In [109]:
def normalize(text):
    doc = nlp(text)
    words = [word.lemma for sent in doc.sentences for word in sent.words]
    return words

In [110]:
from collections import defaultdict, Counter

In [111]:
train_asp['norm_mention'] = [tuple(normalize(m)) for m in train_asp['mention']]

Строим частотный словарь "токенизированное упоминание + категория"

Категория - аспектная категория или тональность

In [139]:
def get_mention_category(data, cat_type):
    mention_categories = data.value_counts(subset=['norm_mention', cat_type])
    mention_categories_dict = defaultdict(dict)
    for key, value in mention_categories.items():
        mention_categories_dict[key[0]][key[1]] = value
    return {k: Counter(v).most_common(1)[0][0] for k, v in mention_categories_dict.items()}

In [140]:
best_mention_cat = get_mention_category(train_asp, 'category')

In [141]:
best_mention_cat

{('ресторан',): 'Whole',
 ('интерьер',): 'Interior',
 ('обслуживание',): 'Service',
 ('официант',): 'Service',
 ('заведение',): 'Whole',
 ('кухня',): 'Food',
 ('место',): 'Whole',
 ('блюдо',): 'Food',
 ('еда',): 'Food',
 ('цена',): 'Price',
 ('персонал',): 'Service',
 ('официантка',): 'Service',
 ('пиво',): 'Food',
 ('порция',): 'Food',
 ('меню',): 'Food',
 ('атмосфера',): 'Interior',
 ('музыка',): 'Interior',
 ('девушка',): 'Service',
 ('сервис',): 'Service',
 ('обстановка',): 'Interior',
 ('администратор',): 'Service',
 ('встретить',): 'Service',
 ('мясо',): 'Food',
 ('ждать',): 'Service',
 ('вечер',): 'Whole',
 ('горячий',): 'Food',
 ('зал',): 'Interior',
 ('принести',): 'Service',
 ('десерт',): 'Food',
 ('кафе',): 'Whole',
 ('салат',): 'Food',
 ('обслуживал',): 'Service',
 ('вино',): 'Food',
 ('поесть',): 'Food',
 ('напиток',): 'Food',
 ('салаты',): 'Food',
 ('счет',): 'Price',
 ('живой', 'музыка'): 'Interior',
 ('заказ',): 'Service',
 ('молодой', 'человек'): 'Service',
 ('стол',):

In [142]:
best_mention_sentiment = get_mention_category(train_asp, 'sentiment')

In [143]:
best_mention_sentiment

{('интерьер',): 'positive',
 ('ресторан',): 'positive',
 ('обслуживание',): 'positive',
 ('кухня',): 'positive',
 ('официант',): 'positive',
 ('место',): 'positive',
 ('заведение',): 'positive',
 ('персонал',): 'positive',
 ('еда',): 'positive',
 ('цена',): 'positive',
 ('блюдо',): 'positive',
 ('атмосфера',): 'positive',
 ('музыка',): 'positive',
 ('обстановка',): 'positive',
 ('порция',): 'positive',
 ('девушка',): 'positive',
 ('пиво',): 'positive',
 ('меню',): 'positive',
 ('официантка',): 'negative',
 ('администратор',): 'positive',
 ('вечер',): 'positive',
 ('встретить',): 'positive',
 ('сервис',): 'positive',
 ('зал',): 'positive',
 ('ждать',): 'negative',
 ('обслуживал',): 'positive',
 ('напиток',): 'neutral',
 ('кафе',): 'positive',
 ('горячий',): 'neutral',
 ('стол',): 'positive',
 ('молодой', 'человек'): 'positive',
 ('поесть',): 'positive',
 ('мясо',): 'positive',
 ('живой', 'музыка'): 'positive',
 ('салаты',): 'positive',
 ('десерт',): 'positive',
 ('принести',): 'negative

In [114]:
dev_texts = pd.read_csv('dev_reviews.txt', delimiter='\t', names=['text_id', 'text'])

In [116]:
dev_texts

,text_id,text,norm_text
0,1751,"Ну вот посетила Чаплин-холл, поделюсь впечатле...","[ну, вот, посетить, Чаплин, -, холл, ,, делить..."
1,10825,Побывали впервые в День всех влюбленных в этом...,"[побывали, впервые, в, день, весь, влюбленных,..."
2,12203,Посетил данный ресторан в начале сентября в ко...,"[посетил, данный, ресторан, в, начало, сентябр..."
3,37220,"Кухня супер, спасибо поварам, интерьер приятны...","[кухня, супер, ,, спасибо, поварам, ,, интерье..."
4,9662,"Были вчера компанией 6 человек в ""Суп-вино"". С...","[быть, вчера, компания, 6, человек, в, "", суп-..."
...,...,...,...
66,9776,Послушав отзывы от друзей мы с женой зашли в э...,"[послушав, отзыв, от, друг, мы, с, жена, зайти..."
67,7011,Отмечали в этом ресторане свадьбу 11 июня 2011...,"[отмечали, в, это, ресторан, свадьба, 11, июнь..."
68,34228,Были в эту субботу. Шли с прекрасным настроени...,"[быть, в, этот, суббота, ., слать, с, прекрасн..."
69,18372,Ресторанчик очень милый.бывала там не раз!Гото...,"[ресторанчик, очень, милый, ., бывать, там, не..."


Длины упоминаний аспектов в трейне:

In [117]:
Counter([len(x) for x in best_mention.keys()])

Counter({1: 609, 2: 317, 3: 160, 4: 50, 5: 27, 12: 1, 7: 5, 6: 9, 9: 1, 8: 1})

Будем учитывать только упоминания длиной 1-5 токенов:

In [162]:
def label_texts(text, mentions, sentiments, max_len=5):
    tokenized = [word for sent in nlp(text).sentences for word in sent.words]
    text_end = len(tokenized)
    for i, token in enumerate(tokenized):
        for l in reversed(range(max_len)):
            if i + l > text_end:
                continue
            span = tokenized[i:i + l]
            key = tuple([t.lemma for t in span])
            if key in mentions:
                start, end = span[0].start_char, span[-1].end_char
                yield mentions[key], text[start:end], start, end, sentiments[key]
                break

Применяем частотные данные к текстам из dev:

In [165]:
with open('dev_pred_aspects.txt', 'w') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for asp in label_texts(text, best_mention_cat, best_mention_sentiment):
            print(idx, *asp, sep="\t", file=f)

# Baseline 3

Посчитаем упоминания аспектов с предсказанной тональностью, припишем
- `absence` - если нет упоминаний данной категории
- `both` - если есть упоминания с разной тональностью
- `positive/neutral/negative` - если все упоминания одной тональности

In [151]:
CATEGORIES = ['Whole', 'Interior', 'Service', 'Food', 'Price']

In [171]:
def get_full_sentiment(text, mentions, sentiment, max_len=5):
    asp_counter = defaultdict(Counter)
    for asp in label_texts(text, best_mention_cat, best_mention_sentiment, max_len):
        category, *_, sentiment = asp
        asp_counter[category][sentiment] += 1
    for c in CATEGORIES:
        if not asp_counter[c]:
            s = 'absence'
        elif len(asp_counter[c]) == 1:
            s = asp_counter[c].most_common(1)[0][0]
        else:
            s = 'both'
        yield c, s

In [172]:
with open('dev_pred_cats.txt', 'w') as f:
    for text, idx in zip(dev_texts['text'], dev_texts['text_id']):
        for c, s in get_full_sentiment(text, best_mention_cat, best_mention_sentiment):
            print(idx, c, s, sep="\t", file=f)